<a href="https://colab.research.google.com/github/anubhavkumar47/Neural-Network-/blob/main/GAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#this code is for generative adversial network on mnist data set

In [9]:
#all requried libarary
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Reshape,LeakyReLU,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train = (x_train-127.5)/127.5
input_vector_size=100

In [11]:
#making the generator network
def build_generator():
  model =Sequential([
      Dense(128,input_dim =input_vector_size),
      LeakyReLU(0.2),
      Dense(784,activation='tanh'),
      Reshape((28,28)) #for 28*28 image
  ])
  return model

def build_discrimanaior():
  model = Sequential([
      Flatten(input_shape=(28,28)),
      Dense(128),
      LeakyReLU(0.2),
      Dense(1,activation='sigmoid')
  ])
  return model





In [12]:
#compling the discrimanator
discrimanator = build_discrimanaior()
discrimanator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])



In [13]:
#build and compile the complete model
generator = build_generator()

discrimanator.trainable =False

gan_input =tf.keras.Input(shape=(100,))

gen_image=generator(gan_input)
gan_output = discrimanator(gen_image)

combinded_model =tf.keras.Model(gan_input,gan_output)
combinded_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
#gan Training

def train_gan(epochs=100, batch_size=128):
  half_batch = int(batch_size / 2)
  for epoch in range(epochs):
    idx = np.random.randint(0, x_train.shape[0], half_batch)
    real_images = x_train[idx]

    noise = np.random.normal(0, 1, (half_batch, 100))
    fake_images = generator.predict(noise)

    d_fake_loss = discrimanator.train_on_batch(fake_images, np.zeros((half_batch, 1)))
    d_real_loss = discrimanator.train_on_batch(real_images, np.ones((half_batch, 1)))
    d_loss = 0.5 * np.add(d_fake_loss, d_real_loss)

    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = combinded_model.train_on_batch(noise, np.ones((batch_size, 1)))

    print(f"Epoch {epoch+1}/{epochs} - Discriminator Loss: {d_loss[0]} - Generator Loss: {g_loss}")

    if epoch % 500 == 0:
      plt.imshow(fake_images[0], cmap='gray')
      plt.title(epoch)
      plt.axis('off')
      plt.show()
